# Análise de Cesto de Compras

Neste exemplo, iremos apresentar uma análise genérica sobre *Market Basket Analysis*.

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(arules)
library(visNetwork)

## Carregar dados

In [ ]:
dados <- read_csv('/home/vm-data-science/dados/dados_mba_exemplo_1.csv')

In [ ]:
dados %>% 
    head()

## Análise exploratória

In [ ]:
dados %>% 
  count(ID)

In [ ]:
dados %>% 
  count(Products)

Observamos as compras de 15 clientes, sendo 15 produtos distribuídos entre os clientes.

## Análise das cestas

Primeiramente buscamos criar a lista de compras para cada cliente.

In [ ]:
lista_compras <- split( dados$Products, dados$ID)
lista_compras

Iremos utilizar o pacote arules. Desta forma, precisamos ajustar os dados para uso desta ferramenta.

In [ ]:
dados_arules <- as( lista_compras, 'transactions' )

Um breve sumário nos mostra, por exemplo, os produtos mais frequentes.

In [ ]:
summary(dados_arules)

Os dados ficam armazenados em formato de matriz, sendo as linhas os produtos e as colunas os clientes.

In [ ]:
as.matrix( dados_arules@data ) * 1

In [ ]:
dados %>% 
    filter(ID == 1)

Podemos obter algumas estatísticas básicas como:

- Frequência relativa dos produtos

In [ ]:
itemFrequency( dados_arules, type = 'relative' )

- Frequência absoluta

In [ ]:
itemFrequency( dados_arules, type = 'absolute' )

- Gráfico dos primeiros produtos mais frequentes.

In [ ]:
itemFrequencyPlot( dados_arules, 
                  topN = 5, 
                  type = 'relative' )

### Criação das regras LHS e RHS

O modelo de análise de cesto de compras busca criar regras de associação entre os produtos.

As regras LHS são do tipo: "Se o cliente compra o produto A e B".

As regras RHS complementam as LHS e são do tipo: "Então compra o produto W".

- Devemos controlar os parâmetros supp (*support*), conf (*confidence*), minlen (número mínimo de regras) e maxlen (número máximo de regras)

In [ ]:
regras <- apriori( dados_arules,
         parameter = list( supp = 0.2,
                           conf = 0.8,
                           minlen = 3,
                           maxlen = 5,
                           target = 'rules') )

Por meio dos parâmetros usados foram obtidas 181 regras de associação.

In [ ]:
as( regras, 'data.frame' ) %>% 
    as_data_frame() %>% 
    head()

Um passo importante também é ajustar as regras redundantes.

Vejamos o exemplo a seguir:

Existem 29 regras contendo os produtos E e F.

In [ ]:
as( regras, 'data.frame' ) %>% 
  dplyr::as_data_frame() %>% 
  mutate( rules = as.character(rules) ) %>% 
  filter( str_detect( rules, 'Product E,Product F' ) )

Porém, 27 delas são redundantes, dado que podem ser estar sendo repetidas em outras regras.

In [ ]:
as( regras[is.redundant(regras)], 'data.frame' ) %>% 
  dplyr::as_data_frame() %>% 
  mutate( rules = as.character(rules) ) %>% 
  filter( str_detect( rules, 'Product E,Product F' ) )

Deste modo, somente 2 regras podem ser vistas como regras únicas.

In [ ]:
as( regras[!is.redundant(regras)], 'data.frame' ) %>% 
  dplyr::as_data_frame() %>% 
  mutate( rules = as.character(rules) ) %>% 
  filter( str_detect( rules, 'Product E,Product F' ) )

Ajustamos todas as regras:

In [ ]:
regras_ajustadas <- regras[!is.redundant(regras)]

Podemos fazer alguns ajustes para deixar as regras num formato melhor.

In [ ]:
regras_ajustadas <- as( regras_ajustadas, 'data.frame' ) %>% 
  dplyr::as_data_frame() %>% 
  mutate( rules = as.character(rules) ) %>% 
  separate( rules, c('regras_lhs', 'regras_rhs'), sep = '=>' ) %>% 
  mutate( regras_lhs = str_replace_all( string = regras_lhs,
                                        pattern = '\\{',
                                        replacement = ''),
          regras_lhs = str_replace_all( string = regras_lhs,
                                        pattern = '\\}',
                                        replacement = ''),
          regras_lhs = str_trim( regras_lhs ),
          regras_rhs = str_replace_all( string = regras_rhs,
                                        pattern = '\\{',
                                        replacement = ''),
          regras_rhs = str_replace_all( string = regras_rhs,
                                        pattern = '\\}',
                                        replacement = ''),
          regras_rhs = str_trim( regras_rhs ) )

In [ ]:
regras_ajustadas %>% 
    head()

- Podemos ordenar pelo *lift*

In [ ]:
regras_ajustadas %>% 
    arrange( desc(lift) ) %>% 
    head()

- Podemos analisar as regras que levam a recomendação de algum produto, como, por exemplo, o produto K.

In [ ]:
regras_ajustadas %>% 
  filter( str_detect( regras_rhs, 'Product K') ) %>% 
  arrange( desc(lift) )

## Gráfico de rede dos produtos

Podemos gerar uma visualização das regras criadas.

- Criação dos nós

In [ ]:
nodes <- regras_ajustadas %>% 
  distinct( regras_rhs ) %>%
  rename( title = regras_rhs) %>% 
  mutate( id = title,
          label = title,
          group = 'A' ) %>%
  bind_rows(., regras_ajustadas %>% 
               distinct( regras_lhs ) %>%
               rename( title = regras_lhs) %>% 
              mutate( id = title,
                      label = "",
                      group = 'B' ) ) 

In [ ]:
nodes %>% 
    head()

- Criação dos *edges*.

In [ ]:
edges <- regras_ajustadas %>% 
  rename( from = regras_lhs,
          to = regras_rhs ) %>% 
  mutate( value = lift,
          weight = lift)

In [ ]:
edges %>% 
    head()

- Criação da rede.

In [ ]:
visNetwork(nodes, 
           edges, 
           width = "100%", 
           height = "850",
           main = "Rede Produtos" ) %>% 
  visEdges(arrows = 'to') %>% 
  visPhysics( solver = "forceAtlas2Based", 
              forceAtlas2Based = list(gravitationalConstant = -60) ) %>% 
  visInteraction( multiselect = TRUE) %>%
  visOptions( highlightNearest = list(enabled =TRUE, degree = 1),
              nodesIdSelection = TRUE ) %>% 
  visLayout(randomSeed = 123) %>% 
  visGroups( groupname = "A", color = list( background = '#FA8072',
                                            border = '#B22222' ) )